### Hedging: Volatility Mismatch

Consider a short position in a European call option on a non-dividend paying stock with a maturity of one year and strike K = 99 EUR. Let the one-year risk-free interest rate be 6% and the current stock price be 100 EUR. Furthermore, assume that the volatility is 20%.

Use the Euler method to perform a hedging simulation.

In [1]:
import numpy as np
from scipy.stats import norm

#### Roadmap

We are short a European call option with $K=99, T = 1$ on a stock with $S_0 = 100, r = 0.06, \sigma = 0.20$. We need to hedge this short option position to manage risk.

The stock evolves according to GBM, discretized via the Euler scheme. The Black-Scholes price dynamics under the risk neutral measure are:

$$
dS_t = rS_t dt + \sigma S_t dW_t
$$

Following the Euler scheme, the discretization of the model is given by:

$$
\hat{S}_{t+\Delta t} = \hat{S}_t + r \hat{S}_t \Delta t + \sigma \hat{S}_t \sqrt{\Delta t} Z
$$

where $\Delta t = T / N$ depends on how frequently we rebalance. 

At each rebalancing date, the Black-Scholes call price is given by:

$$
C_{BS}(S_t, K, t, T, \sigma) = S_t N(d_+) - Ke^{-r(T-t)} N(d_-)
$$

where 

$$
d_{\pm} = \frac{\ln \frac{S_t}{K} + (r \pm \frac{\sigma^2}{2})(T - t)}{\sigma \sqrt{T - t}}
$$

We hedge the short option position at time $\Delta t$ by buying shares, where:

$$
\Delta_t = \frac{\partial C}{\partial S}(t, S_t) = N(d_1)
$$

This choice removes the linear term in $\delta S$. We reassess and rebalance at each subsequent time $t + \Delta t$.

The total PnL over a short interval $\delta t$ for the short option, long $\Delta t$ shares position is:

$$
\text{PnL} = -[C(t + \delta t, S_{t+\delta t}) - C(t, S_t)] + rC(t, S_t) \delta t + \Delta_t(\delta S - r S_t \delta t)
$$

Expanding $C(t + \delta t, S_{t+\delta t}$ to second order, we get the carry PnL:

$$
\text{PnL} = -A(t, S_t) \delta t - B(t, S_t) \left( \frac{\delta S}{S_t} \right)^2
$$

where

$$
A(t, S_t) = \frac{\partial C}{\partial t} - rC + r S_t \frac{\partial C}{\partial S}, \quad B(t, S_t) = \frac{1}{2} S_t^2 \frac{\partial^2 C}{\partial S^2}
$$

A natural risk management condition is to require that our hedged portfolio neither consistently gains or loses money on average. Formally, this condition implies:

$$
A(t, S) = -\hat{\sigma}^2 B(t, S), \quad \text{for all } t, S.
$$

From the previous P&L equation, we now get the simplified expression:

$$
\text{PnL} = -\frac{1}{2} S^2 \frac{\partial^2 C_{\hat{\sigma}}}{\partial S^2} \left[ \left( \frac{\delta S}{S} \right)^2 - \hat{\sigma}^2 \delta t \right]
$$



#### 1. Matching Volatility

Conduct an experiment where the volatility in the stock price process matches the volatility used in the delta computation (i.e., both set to 20%). Vary the frequency of hedge adjustments (from daily to weekly) and explain the results.

In [2]:
# Parameters

S0 = 100 # Spot price
K = 99 # strike price
T = 1.0 # maturity
r = 0.06 # risk-free rate
sigma = 0.20 # volatility (real same as implied for task 1)

def get_call_price(S, t, K, T, r, sigma):
    """Black-Scholes European call price (eq. 91)"""
    tau = T - t
    if tau <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    return S * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)

def get_delta(S, t, K, T, r, sigma):
    """Black Scholes delta = N(d1) (eq.97 + Greeks table)"""
    tau = T - t
    if tau <= 0:
        return 1.0 if S > K else 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    return norm.cdf(d1)
    

In [4]:
def hedge(S0, T, K, r, sigma_r, sigma_i, N_steps, seed = None):
    """
    Simulate one delta-hedge path.
    
    sigma_r: volatility driving the stock
    sigma_i: volatility used for pricing/hedging (Black Scholes)
    
    """
    if seed is not None:
        np.random.seed(seed)
        
    dt = T / N_steps
    S = np.zeros(N_steps + 1)
    S[0] = S0
    
    # Simulate stock path via Euler scheme
    Z = np.random.randn(N_steps)
    for i in range(N_steps):
        S[i+1] = S[i] + r * S[i] * dt + sigma_r * S[i] * np.sqrt(dt) * Z[i]
        
    # Init hedge
    delta_0 = get_delta(S[0], 0, K, T, r, sigma_i)
    C0 = get_call_price(S[0], 0, K, T, r, sigma_i)
    B = C0 - delta_0 * S[0] # Bank acccount
    delta_old = delta_0
    
    # Rebalance at each step
    for i in range(1, N_steps):
        t_i = i * dt
        B = B * np.exp(r * dt)
        delta_new = get_delta(S[i], t_i, K, T, r, sigma_i)
        B = B - (delta_new - delta_old) * S[i]           # Fund rebalancing
        delta_old = delta_new
        
    # Final PnL at maturity
    B = B * np.exp(r * dt)  
    payoff = max(S[-1] - K, 0)   # Option settlement
    PnL = B + delta_old * S[-1] - payoff
    
    return PnL, S

In [ ]:
results_1 = hedge(...)

#### Interpretation

#### 2. Mismatched Volatility

Perform numerical experiments where the volatility in the stock price process does not match the volatility used in the delta valuation. Run computational experiments for various levels of volatility and discuss the outcomes.

#### Interpretation